<style type="text/css" media="screen">
a:link { color:#FFFFFF; text-decoration: none; }
a:visited { color:#FFFFFF; text-decoration: none; }
a:hover { color:#FFFFFF; text-decoration: none; }
a:active { color:#FFFFFF; text-decoration: underline; }
</style>
# <a id="sc" style="text-decoration:none;">Managing HPE Cloud Volumes with API</a>
![](Pictures/hpedevlogo-NB.JPG)

## Part 1 - Explore HPE Cloud Volumes

<br>
<br>
<br>
<br>



## If you haven't red the **introduction yet**, click the "Back" button below.

 <a href="0-ReadMeFirst.ipynb" target="New" title="Back: Initialization section"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#FFAD33;color:#fff;position:relative;width:10%; height: 30px;float: left;"><b>Back</b></button></a>
 </br>

## <i class="fas fa-2x fa-cogs" style="color:#FFAD33;"></i>&nbsp;&nbsp;Start the lab

</br>
<p><i class="fas fa-2x fa-walking" style="color:#FFAD33;"></i>&nbsp;&nbsp;<b>STEP 1: </b>Get your connection token</p>

<div class="alert" style="background-color:#FFAD33;color:#fff;" role="alert">
  
  <p><i class="fas fa-eye"></i>&nbsp;&nbsp;Run the command below, it will generate a connection token for you. it may take up to 30 seconds to complete in case of high concurrent requests. Once you got the token print it. </p>
    <hr>
    <p class="mb-0"><p class="mb-0"><i class="fas fa-sync-alt"></i>&nbsp;&nbsp;Run the command in the next Cells by clicking - click once per cell - <i class="fas fa-play"></i></p>
 
</div>

In [ ]:
TOKEN=`$HOME/get-cloudvol-token.shc.x`

In [ ]:
echo ${TOKEN}

In [ ]:
curl -s --location --request GET 'https://demo.cloudvolumes.hpe.com/api/v2/session' \
--header "X-Auth-Token: ${TOKEN}" \
--header 'Content-Type: application/json'

</br>

**jq my friend**</br>
We've got a result in JSON format. However, this result is shipped back in a single line which make it very hard to read.</br>
it is then the right time to introduce `jq` which is a command-line JSON processor. From this time, we will pipe the cURL result into a jq (sometime customized) command.</br>
To know more about this tool, I recommend the post below:</br>
- https://shapeshed.com/jq-json/


<div class="alert" style="background-color:#FFAD33;color:#fff;" role="alert">
  
  <p><i class="fas fa-eye"></i>&nbsp;&nbsp;Rerun the same cURL command which is piped to jq this time. </p>
    <hr>
    <p class="mb-0"><p class="mb-0"><i class="fas fa-sync-alt"></i>&nbsp;&nbsp;Run the command in the next Cells  by clicking <i class="fas fa-play"></i></p>
 
</div>

In [ ]:
curl -s --location --request GET 'https://demo.cloudvolumes.hpe.com/api/v2/session' \
--header "X-Auth-Token: ${TOKEN}" \
--header 'Content-Type: application/json' | jq

</br>
<p><i class="fas fa-2x fa-walking" style="color:#FFAD33;"></i>&nbsp;&nbsp;<b>STEP 2: </b>Get the replication store</p>

The replication store is the logical unit hosting the the replicas of HPE Nimble volumes.</br>
In a replication store are defined information like the cloud region, the capacity of the store and the performance profile

In [ ]:
curl -s --location --request GET 'https://demo.cloudvolumes.hpe.com/api/v2/replication_stores' \
--header 'Content-Type: application/json' \
--header "X-Auth-Token: ${TOKEN}" \
--data-raw '' | jq

</br>

Among the data sent, one can isolate the replication store ids. </br>

In [ ]:
RepStore=$(curl -s --location --request GET 'https://demo.cloudvolumes.hpe.com/api/v2/replication_stores' \
--header 'Content-Type: application/json' \
--header "X-Auth-Token: ${TOKEN}" \
--data-raw '' | jq '. | .[] | .[0] | .id')
echo "Replication Store id is $RepStore"

</br>

As we may have several replication stores for the same HPE Cloud Volumes account, one can restrict the feedback to a unique store.</br>
For that, let's define a variable containing the replication store id and target the API call to this specific object.</br>

In [ ]:
curl -s --location --request GET "https://demo.cloudvolumes.hpe.com/api/v2/replication_stores/${RepStore}" \
--header 'Content-Type: application/json' \
--header "X-Auth-Token: ${TOKEN}" \
--data-raw '' | jq

</br>
<p><i class="fas fa-2x fa-walking" style="color:#FFAD33;"></i>&nbsp;&nbsp;<b>STEP 3: </b>Look at the replication volumes</p>

</br>

A replication store contains one or several replication volumes. </br>
The query below will list these volumes. </br>

In [ ]:
curl -s --location --request GET "https://demo.cloudvolumes.hpe.com/api/v2/replication_stores/${RepStore}/replica_volumes" \
--header 'Content-Type: application/json' \
--header "X-Auth-Token: ${TOKEN}" \
--data-raw '' | jq

As we did for the replication stores, we can identify the replication volumes with their id.</br>

In [ ]:
RepVolume=$(curl -s --location --request GET "https://demo.cloudvolumes.hpe.com/api/v2/replication_stores/${RepStore}/replica_volumes" \
--header 'Content-Type: application/json' \
--header "X-Auth-Token: ${TOKEN}" \
--data-raw '' | jq -r '. | .[] | .[] | .id'| tail -1)
echo 
echo "Replication Volume id is $RepVolume"

</br>

With this new variable, we will be able to get detailed information about the replication volume.</br>

In [ ]:
curl -s --location --request GET "https://demo.cloudvolumes.hpe.com/api/v2/replication_stores/${RepStore}/replica_volumes/${RepVolume}" \
--header 'Content-Type: application/json' \
--header "X-Auth-Token: ${TOKEN}" \
--data-raw '' | jq

</br>

## Note: The following  cell is OPTIONAL!
The replication volume contains snapshots taken at a regular basis.</br>
In the query below, we will sort the snapshots per date of creation. To identify the sorting, look after the `creation_date` value which is expressed in timestamp second.</br>
`The list may be fairly long.` </br>

In [ ]:
curl -s --location --request GET "https://demo.cloudvolumes.hpe.com/api/v2/replication_stores/${RepStore}/replica_volumes/${RepVolume}/snapshots?sort=creation_time" \
--header 'Content-Type: application/json' \
--header "X-Auth-Token: ${TOKEN}" \
--data-raw '' | jq

</br>

In the next code sequence, we will identify the most recent volume. It will be our source for the cloning activity to be done during the section 2.</br>

In [ ]:
SnapRefArray=`curl -s --location --request GET "https://demo.cloudvolumes.hpe.com/api/v2/replication_stores/${RepStore}/replica_volumes/${RepVolume}/snapshots?sort=creation_time" \
--header 'Content-Type: application/json' \
--header "X-Auth-Token: ${TOKEN}" \
--data-raw '' | jq -r '. | .[] | .[] | .id' `
for i in ${SnapRefArray}
do
  SnapRef=${i}
done
echo ${SnapRef}

</br>
<p><i class="fas fa-2x fa-walking" style="color:#FFAD33;"></i>&nbsp;&nbsp;<b>STEP 3: </b>List the existing Block Volumes</p>


The Block Volume is the entity which is presented to the target VM via an iSCSI protocol already established.</br>
The query below will list the existing Block Volumes.</br>

In [ ]:
curl -s --location --request GET 'https://demo.cloudvolumes.hpe.com/api/v2/cloud_volumes' \
--header 'Content-Type: application/json' \
--header "X-Auth-Token: ${TOKEN}" \
--data-raw ''

</br>

The returned list can be long. This is way it was remained intentionnoly in a single line.</br>
The reason for this size is because all block volumes are kept recoreded even those which were - logically - deleted.</br>
In the next query, we will restrict the view to the active volumes (note the `?filters=marked_for_deletion=0` parameter at the end of query).</br>

In [ ]:
curl -s --location --request GET 'https://demo.cloudvolumes.hpe.com/api/v2/cloud_volumes?filters=marked_for_deletion=0' \
--header 'Content-Type: application/json' \
--header "X-Auth-Token: ${TOKEN}" \
--data-raw '' | jq

</br>
<p><i class="fas fa-2x fa-walking" style="color:#FFAD33;"></i>&nbsp;&nbsp;<b>STEP 4: </b>Backup your environment data</p>

</br>

Before you continue with the next section, run the code below to backup your environment.</br>

In [ ]:
filename="cv-lab.tmp"
rm -f $HOME/${filename}
echo "TOKEN " ${TOKEN} >> $HOME/${filename}
echo "RepVolume " ${RepVolume} >> $HOME/${filename}
echo "RepStore " ${RepStore} >> $HOME/${filename}
echo "SnapRef "  ${SnapRef} >> $HOME/${filename}
cat $HOME/${filename}

<br><br>

## <i class="fas fa-2x fa-map-marker-alt" style="color:#FFAD33;"></i>&nbsp;&nbsp;Next Steps

Manipulate the block volumes.

</br>
 <a href="0-ReadMeFirst.ipynb" target="New" title="Back: Introduction"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#FFAD33;color:#fff;position:relative;width:10%; height: 30px;float: left;"><b>Back</b></button></a>
 <a href="2-WKSHP-Cloud-Vol-Manipulate.ipynb#sc" target="New" title="Next:Persistent Volumes"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#FFAD33;color:#fff;position:relative;width:10%; height: 30px;float: right;"><b>Next</b></button></a>


<h2>Next&nbsp;&nbsp;&nbsp;&nbsp;<a href="2-WKSHP-Cloud-Vol-Manipulate.ipynb#sc" target="New" title="Next: Introduction"><i class="fas fa-chevron-circle-right" style="color:#FFAD33;"></i></a></h2>